In [28]:
import PIL,torch
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torchvision.models import vgg16

In [58]:
class SkinDataset(Dataset):
    folds=5
    def __init__(self, root , num_classes,fold=0,training=False,transform=None):
        self.data_path = []
        self.transform = transform
        self.sides = 112
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((self.sides,self.sides)),
                transforms.RandomRotation(5),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
            ])
        self.training = training
        for label in range(num_classes):
            self.data_dir = os.path.join(root,os.listdir(root)[label])
            self.filename = os.listdir(self.data_dir)
            l = len(self.filename)
            inter = l//SkinDataset.folds
            picked = list(range(inter* fold,inter * (fold+1))) if not training else list(range(0,inter*fold))+list(range(inter*(fold+1),l))

            for i in picked:
                file_path = os.path.join(self.data_dir , self.filename[i])
#                 img = Image.open(fil)
                self.data_path.append((file_path, label))
    
    def __getitem__(self , index):
        ddir , label = self.data_path[index]
        img = Image.open(ddir)
        imgmat = self.transform(img)
        if self.training:
            imgmat += torch.randn(3,self.sides,self.sides)/256
        result = (imgmat, label)
        del imgmat
        del img
        return result
    
    def __len__(self):
        return len(self.data_path)

In [59]:
# del training_dataloader
batch_size = 64
training_dataset = SkinDataset(os.path.join(os.getcwd(),'../Skin40'), 40, training=True)
test_dataset = SkinDataset(os.path.join(os.getcwd(),'../Skin40'), 40)
training_dataloader = DataLoader(training_dataset , batch_size = batch_size , num_workers = 1, shuffle = True)
test_dataloader = DataLoader(test_dataset , batch_size = batch_size , num_workers = 1, shuffle = True)

In [34]:
def train(model, train_loader, loss_func, optimizer, device):
    """
    train model using loss_fn and optimizer in an epoch.
    model: CNN networks
    train_loader: a Dataloader object with training data
    loss_func: loss function
    device: train on cpu or gpu device
    """
    total_loss = 0
    model.train()
    # train the model using minibatch
    for i, (images, targets) in enumerate(train_loader):
        images = images.to(device)
        targets = targets.to(device)
        
        # forward
        outputs = model(images)
        loss = loss_func(outputs, targets)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # every 100 iteration, print loss
        if (i + 1) % 6 == 0:
            print ("Step [{}/{}] Train Loss: {:.4f}"
                   .format(i+1, len(train_loader), loss.item()))
    return total_loss / len(train_loader)

In [7]:
def evaluate(model, val_loader, device, nclasses):
    """
    model: CNN networks
    val_loader: a Dataloader object with validation data
    device: evaluate on cpu or gpu device
    return classification accuracy of the model on val dataset
    """
    # evaluate the model
    model.eval()
    # context-manager that disabled gradient computation
    with torch.no_grad():
        correct = 0
        total = 0
        cm = torch.zeros(nclasses, nclasses)
        for i, (images, targets) in enumerate(val_loader):
            # device: cpu or gpu
            images = images.to(device)
            targets = targets.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            
            
            for t, p in zip(targets.view(-1), predicted.view(-1)):
                cm[t.long()][p.long()] +=1
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
            
        accuracy = correct / total
        print('Accuracy on Test Set: {:.4f} %'.format(100 * accuracy))
        show_heatmap(cm, nclasses)
        return accuracy

In [17]:
import matplotlib.pyplot as plt
def show_curve(ys, title):
    """
    plot curlve for Loss and Accuacy
    Args:
        ys: loss or acc list
        title: loss or accuracy
    """
    x = np.array(range(len(ys)))
    y = np.array(ys)
    plt.plot(x, y, c='b')
    plt.axis()
    plt.title('{} curve'.format(title))
    plt.xlabel('epoch')
    plt.ylabel('{}'.format(title))
    plt.show()

def show_heatmap(cm, nclasses):
    labels = range(nclasses)
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    ax.set_yticks(labels)
    ax.set_yticklabels(labels)
    ax.set_xticks(labels)
    ax.set_xticklabels(labels)
    
    im = ax.imshow(cm, cmap=plt.cm.hot_r )
    plt.colorbar(im)
    plt.title("Confusioin Matrix")
    plt.show()
    print('Accuracy of each class:\n', cm.diag()/(cm.sum(1)+1e-6))
    print('Recall of each class:\n', cm.diag()/(cm.sum(0)+1e-6))

In [9]:
def fit(model, num_epochs, optimizer, schedulr, device,nclasses):
    """
     train and evaluate an classifier num_epochs times.
    n and evaluate an classifier num_epochs times.
    We use optimizer and cross entropy loss to train the model. 
    Args: 
        model: CNN network
        num_epochs: the number of training epochs
        optimizer: optimize the loss function    loss_func.to(device)
    loss_func.to(device)
        schedulr: scheduling learning rate

    """
    # loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    
    model.to(device)
    loss_func.to(device)
    
    # log train loss and test accuracy
    losses = []
    accs = []
    
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}: , lr = {}'.format(epoch + 1, num_epochs , optimizer.param_groups[0]['lr']))
        # train step
        loss = train(model, training_dataloader, loss_func, optimizer, device)
        losses.append(loss)
        schedulr.step()
        
        # evaluate step
        accuracy = evaluate(model, test_dataloader, device,nclasses)
        accs.append(accuracy)
        
    
    # show curve
    show_curve(losses, "train loss")
    show_curve(accs, "test accuracy")

In [53]:
import os
wd = os.getcwd()
os.chdir(os.path.join(os.getcwd(),'../SAN'))
print(os.getcwd())

/home/jovyan/Documents/dataset_skin40/SAN


In [54]:
modelbest=torch.load('san19_pairwise/model/model_best.pth')
# sdict = modelbest['state_dict']
# for k,v in modelbest['state_dict'].items():
#     print(type(k),k, v.shape)

In [55]:
from model import san
net = san(sa_type=0, layers=(3, 3, 4, 6, 3), kernels=[3, 7, 7, 7, 7], num_classes=1000)
net.load_state_dict({k[7:]:v for k,v in modelbest['state_dict'].items()})
# net = modelbest
net.fc = torch.nn.Sequential(nn.Linear(2048, 4096),
                             nn.Sigmoid(),
                             nn.Dropout(p=0.6),
                             nn.Linear(4096, 10)
                            )
for k,v in net.state_dict().items():
    print(k, v.shape)

conv_in.weight torch.Size([64, 3, 1, 1])
bn_in.weight torch.Size([64])
bn_in.bias torch.Size([64])
bn_in.running_mean torch.Size([64])
bn_in.running_var torch.Size([64])
bn_in.num_batches_tracked torch.Size([])
conv0.weight torch.Size([64, 64, 1, 1])
bn0.weight torch.Size([64])
bn0.bias torch.Size([64])
bn0.running_mean torch.Size([64])
bn0.running_var torch.Size([64])
bn0.num_batches_tracked torch.Size([])
layer0.0.bn1.weight torch.Size([64])
layer0.0.bn1.bias torch.Size([64])
layer0.0.bn1.running_mean torch.Size([64])
layer0.0.bn1.running_var torch.Size([64])
layer0.0.bn1.num_batches_tracked torch.Size([])
layer0.0.sam.conv1.weight torch.Size([4, 64, 1, 1])
layer0.0.sam.conv1.bias torch.Size([4])
layer0.0.sam.conv2.weight torch.Size([4, 64, 1, 1])
layer0.0.sam.conv2.bias torch.Size([4])
layer0.0.sam.conv3.weight torch.Size([16, 64, 1, 1])
layer0.0.sam.conv3.bias torch.Size([16])
layer0.0.sam.conv_w.0.weight torch.Size([6])
layer0.0.sam.conv_w.0.bias torch.Size([6])
layer0.0.sam.conv_

In [56]:
os.chdir(wd)
print(os.getcwd())

/home/jovyan/Documents/dataset_skin40/SYSUDeepLearningSkin


In [57]:
# Hyper-parameters
num_epochs = 20
lr =1e-5
nclasses = 40
feature_tune=False
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


cost = torch.nn.CrossEntropyLoss()
for parma in net.parameters():
    parma.requires_grad = feature_tune
for param in net.fc.parameters():
    param.requires_grad = True
# # optimizer
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
# schedulr = torch.optim.lr_scheduler.StepLR(optimizer , step_size = 1 , gamma = 0.9)
for name,param in net.named_parameters():
    print(param.requires_grad, name)

False conv_in.weight
False bn_in.weight
False bn_in.bias
False conv0.weight
False bn0.weight
False bn0.bias
False layer0.0.bn1.weight
False layer0.0.bn1.bias
False layer0.0.sam.conv1.weight
False layer0.0.sam.conv1.bias
False layer0.0.sam.conv2.weight
False layer0.0.sam.conv2.bias
False layer0.0.sam.conv3.weight
False layer0.0.sam.conv3.bias
False layer0.0.sam.conv_w.0.weight
False layer0.0.sam.conv_w.0.bias
False layer0.0.sam.conv_w.2.weight
False layer0.0.sam.conv_w.3.weight
False layer0.0.sam.conv_w.3.bias
False layer0.0.sam.conv_w.5.weight
False layer0.0.sam.conv_w.5.bias
False layer0.0.sam.conv_p.weight
False layer0.0.sam.conv_p.bias
False layer0.0.bn2.weight
False layer0.0.bn2.bias
False layer0.0.conv.weight
False layer0.0.conv.bias
False layer0.1.bn1.weight
False layer0.1.bn1.bias
False layer0.1.sam.conv1.weight
False layer0.1.sam.conv1.bias
False layer0.1.sam.conv2.weight
False layer0.1.sam.conv2.bias
False layer0.1.sam.conv3.weight
False layer0.1.sam.conv3.bias
False layer0.1.

In [51]:
num_epochs=50

In [60]:
schedulr = torch.optim.lr_scheduler.StepLR(optimizer , step_size = 1 , gamma = 0.8)
fit(net, num_epochs, optimizer, schedulr,device, nclasses)  #

Epoch 1/20: , lr = 1e-05


RuntimeError: CUDA error: device-side assert triggered

In [20]:
print(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

cuda


In [ ]:
for i,(dat, tar) in enumerate(test_dataloader):
    print(i ,dat.shape, tar)
    if ( i>30):
        break

In [27]:
torch.save({'epoch':30 , 'state_dict':net.state_dict()}, '../baseSAN_chk0.pth')